In [21]:
import pandas as pd
pd.set_option('display.max_columns', 45)

In [22]:
weekly = pd.read_parquet('data/weekly.parquet')
weekly = weekly.loc[weekly.index != 3821]
weekly['headshot_url'] = weekly['headshot_url'].fillna('no_url')

In [23]:
depth = pd.read_csv('data/depth.csv', index_col=0).sort_values(by='depth_team')
off = depth.loc[depth.formation=='Offense'].copy().drop(
    ['pfr_game_id', 'player', 'pfr_player_id', 'opponent',
       'defense_snaps', 'defense_pct',
       'st_snaps', 'st_pct'],axis=1)
off

C:\Users\Sharedw\AppData\Local\Temp\ipykernel_17764\2214932501.py:1: DtypeWarning: Columns (17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  depth = pd.read_csv('data/depth.csv', index_col=0).sort_values(by='depth_team')


,season,team_name,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth,game_id,offense_snaps,offense_pct
0,2010,ARI,1.0,REG,1,Brown,Levi,Levi,Offense,00-0025392,67.0,T,BRO548586,LT,Levi Brown,0.0,NaN,NaN,NaN
303085,2018,DEN,2.0,REG,1,Veldheer,Edwin,Jared,Offense,00-0027674,69.0,T,VEL116302,RT,Jared Veldheer,1.0,2018_02_OAK_DEN,32.0,0.48
303079,2018,DEN,2.0,REG,1,Heuerman,Jeff,Jeff,Offense,00-0032227,82.0,TE,HEU190731,TE,Jeff Heuerman,0.0,2018_02_OAK_DEN,41.0,0.62
303077,2018,DEN,2.0,REG,1,Janovich,Andy,Andy,Offense,00-0032956,41.0,FB,JAN605717,FB,Andy Janovich,0.0,2018_02_OAK_DEN,11.0,0.17
303074,2018,DEN,2.0,REG,1,Leary,Ronald,Ronald,Offense,00-0029010,65.0,G,LEA610932,LG,Ronald Leary,0.0,2018_02_OAK_DEN,66.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447139,2022,CIN,20.0,DIV,3,Asiasi,Devin,Devin,Offense,00-0036194,86.0,TE,ASI024038,TE,Devin Asiasi,2.0,2022_20_CIN_BUF,14.0,0.19
447136,2022,CIN,20.0,DIV,3,Evans,Chris,Chris,Offense,00-0036857,25.0,RB,EVA105967,HB,Chris Evans,2.0,NaN,NaN,NaN
277173,2017,LV,7.0,REG,3,Cook,Connor,Connor,Offense,00-0032893,18.0,QB,COO045872,QB,Connor Cook,2.0,NaN,NaN,NaN
277174,2017,LV,7.0,REG,3,Walford,Clive,Clive,Offense,00-0032256,87.0,TE,WAL062390,TE,Clive Walford,2.0,2017_07_KC_OAK,8.0,0.10


In [24]:
context_cols = ['player_id', 'player_display_name','position',
       'position_group'
]
game_context = ['week','season_type', 'season', 'opponent_team', 'recent_team', 'game_id']
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',]
rush_stat_cols = [
     'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'fantasy_points', 'fantasy_points_ppr',
       'offense_snaps','offense_pct'
]

depth_cols = ['season', 'last_name',
       'first_name', 'football_name', 'formation', 'gsis_id', 'jersey_number',
       'position', 'elias_id', 'depth_position', 'full_name']


In [25]:

qb_stats_eng = [
    "completions",
    "attempts",
    "passing_yards",
    "passing_tds",
    "interceptions",
    "sacks",
    "sack_yards",
    "passing_air_yards",
    "passing_yards_after_catch",
    "passing_first_downs",
    "passing_2pt_conversions",
    "fantasy_points_ppr",
       'offense_snaps','offense_pct',
       #"completion_pct"
]

rush_stats_eng = [
    "carries",
    "rushing_yards",
    "rushing_tds",
    "rushing_first_downs",
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "fantasy_points",
    "fantasy_points_ppr",
       'offense_snaps','offense_pct',
      # 'yards_per_carry',
]
receiver_stats_eng = [
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "receiving_air_yards",
    "receiving_yards_after_catch",
    "receiving_first_downs",
    "target_share",
    "air_yards_share",
    "fantasy_points_ppr",
      'offense_snaps','offense_pct'
]
stats_to_eng = list(set(qb_stats_eng + rush_stats_eng + receiver_stats_eng))

In [26]:
temp = off.merge(
    weekly,
    how="outer",
    left_on=["gsis_id", "team_name", "season", "week", "position"],
    right_on=["player_id", "recent_team", "season", "week", "position"],
).sort_values(by=['season','week']).reset_index(drop=True)
temp[context_cols] = temp.groupby(depth_cols)[context_cols].bfill()
temp[context_cols] = temp.groupby(depth_cols)[context_cols].ffill()


In [27]:
temp.loc[~temp.offense_snaps.isna(), stat_cols] = temp.loc[~temp.offense_snaps.isna(), stat_cols].fillna(0)
player_fantasy = temp.copy(deep=True).sort_values(by=['season','week']) 
roll_columns = [x + '_roll' for x in stats_to_eng]
player_fantasy[roll_columns] = player_fantasy.groupby(context_cols)[stats_to_eng].transform(lambda x: x.ewm(span=7, min_periods=0).mean().shift())
#player_fantasy[roll_columns] = player_fantasy.groupby(context_cols)[roll_columns].shift(1)

In [28]:
player_fantasy['completion_pct_roll'] = player_fantasy['completions_roll'] / player_fantasy['attempts_roll']
player_fantasy['yards_per_carry_roll'] = player_fantasy['rushing_yards_roll'] / player_fantasy['carries_roll']
player_fantasy['yards_per_catch_roll'] = player_fantasy['receiving_yards_roll'] / player_fantasy['receptions_roll']
player_fantasy['yards_per_target_roll'] = player_fantasy['receiving_yards_roll'] / player_fantasy['targets_roll']
receiver_stats_eng += ['yards_per_catch','yards_per_target']
rush_stats_eng += ['yards_per_carry']
qb_stats_eng += ['completion_pct']

In [29]:
player_fantasy.loc[player_fantasy.full_name =='Brock Bowers'].sort_values(by=['season','week'])[['season','week','offense_snaps', 'receptions', 'fantasy_points_ppr', 'yards_per_catch_roll', 'receptions_roll']]

,season,week,offense_snaps,receptions,fantasy_points_ppr,yards_per_catch_roll,receptions_roll
222770,2024,1.0,41.0,6.0,11.800000,NaN,NaN
223585,2024,2.0,43.0,9.0,18.799999,9.666667,6.000000
224398,2024,3.0,42.0,3.0,7.100000,10.481481,7.714286
225207,2024,4.0,48.0,2.0,5.100000,11.209524,5.675676
226017,2024,5.0,55.0,8.0,23.700001,10.920844,4.331429
226824,2024,6.0,50.0,9.0,16.100000,11.491439,5.533931
227629,2024,7.0,66.0,10.0,19.299999,9.994680,6.588061
228436,2024,8.0,53.0,5.0,10.800000,9.730006,7.572445
229232,2024,9.0,55.0,5.0,15.500000,10.108778,6.857787
230016,2024,10.0,NaN,NaN,NaN,9.873005,6.355637


In [30]:
player_fantasy =player_fantasy.groupby(['team_name','season','week','game_id', 'game_type', 'position', 'dense_depth', 'gsis_id']).head(1)

In [31]:
player_fantasy[[x for x in player_fantasy.columns if x not in stat_cols]].drop(
    ["recent_team", "opponent_team"], axis=1
).to_parquet("data/agg/player_weekly_agg.parquet")


In [32]:
agg = player_fantasy.copy()
agg = agg.sort_values(by=['team_name','season','week','full_name'])


In [33]:
def stack_players(df, num_players, index_cols, value_cols):
    'df should be seperated by position already'
    df = df.loc[df.dense_depth < num_players]
    piv = df.pivot(index=index_cols, columns=['position','dense_depth'], values=value_cols).reset_index()
    piv.columns = [x[0] + '_' + x[1] +'_' +  (str(int(x[2]))) if x[2] != '' else x[0] + x[1] for x in piv.columns ]
    return piv

In [34]:
cols_to_agg =  (['gsis_id','full_name'] + roll_columns)

In [35]:
qb_roll_stat_cols = ['gsis_id','full_name'] + [x + '_roll' for x in qb_stats_eng] 
receiver_roll_stat_cols = ['gsis_id','full_name'] + [x + '_roll' for x in receiver_stats_eng] 
rush_stat_cols = ['gsis_id','full_name'] + [x + '_roll' for x in rush_stats_eng]

team_qb_stats = stack_players(agg.loc[(agg.position.isin(['QB'])) & (~agg.week.isna())],2,  ['team_name','season','week', 'game_type'], qb_roll_stat_cols)
wr_temp = stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],5,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)
te_temp = stack_players(agg.loc[(agg.position.isin(['TE'])) & (~agg.week.isna())],3,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)
rb_temp =stack_players(agg.loc[(agg.position.isin(['RB'])) & (~agg.week.isna())],2,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)
team_receiver_stats = wr_temp.merge(te_temp, on=['team_name','season','week', 'game_type'])
team_receiver_stats = team_receiver_stats.merge(rb_temp, on=['team_name','season','week', 'game_type'])
team_rb_stats = stack_players(agg.loc[(agg.position.isin(['RB'])) & (~agg.week.isna())],3,  ['team_name','season','week', 'game_type'], rush_stat_cols)

In [36]:
stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],5,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)

,team_name,season,week,game_type,gsis_id_WR_0,gsis_id_WR_3,gsis_id_WR_1,gsis_id_WR_2,gsis_id_WR_4,full_name_WR_0,full_name_WR_3,full_name_WR_1,full_name_WR_2,full_name_WR_4,receptions_roll_WR_0,receptions_roll_WR_3,receptions_roll_WR_1,receptions_roll_WR_2,receptions_roll_WR_4,targets_roll_WR_0,targets_roll_WR_3,targets_roll_WR_1,...,fantasy_points_ppr_roll_WR_2,fantasy_points_ppr_roll_WR_4,offense_snaps_roll_WR_0,offense_snaps_roll_WR_3,offense_snaps_roll_WR_1,offense_snaps_roll_WR_2,offense_snaps_roll_WR_4,offense_pct_roll_WR_0,offense_pct_roll_WR_3,offense_pct_roll_WR_1,offense_pct_roll_WR_2,offense_pct_roll_WR_4,yards_per_catch_roll_WR_0,yards_per_catch_roll_WR_3,yards_per_catch_roll_WR_1,yards_per_catch_roll_WR_2,yards_per_catch_roll_WR_4,yards_per_target_roll_WR_0,yards_per_target_roll_WR_3,yards_per_target_roll_WR_1,yards_per_target_roll_WR_2,yards_per_target_roll_WR_4
0,ARI,2012,1.0,REG,00-0027691,00-0026221,00-0022921,00-0029638,NaN,Andre Roberts,Early Doucet,Larry Fitzgerald,Michael Floyd,NaN,4.330959,2.331713,5.85086,NaN,NaN,7.802407,5.332241,11.509924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.667271,13.780717,17.756457,NaN,NaN,5.921188,6.026112,9.026171,NaN,NaN
1,ARI,2012,2.0,REG,00-0027691,00-0026221,00-0022921,00-0029638,NaN,Andre Roberts,Early Doucet,Larry Fitzgerald,Michael Floyd,NaN,4.539521,2.539181,5.276371,0.0,NaN,8.175736,4.608204,11.351648,...,0.0,NaN,51.0,30.0,59.0,17.0,NaN,0.82,0.48,0.95,0.27,NaN,10.712844,13.249834,17.284326,NaN,NaN,5.948233,7.300833,8.033946,0.0,NaN
2,ARI,2012,3.0,REG,00-0022921,00-0029638,00-0027691,00-0026221,NaN,Larry Fitzgerald,Michael Floyd,Andre Roberts,Early Doucet,NaN,4.024622,0.0,3.500291,2.674086,NaN,9.492438,0.428571,6.068886,...,6.185825,NaN,60.714286,10.142857,52.714286,28.285714,NaN,0.967143,0.161429,0.842857,0.451429,NaN,16.31815,NaN,9.982003,11.197194,NaN,6.9186,0.0,5.757221,6.758744,NaN
3,ARI,2012,4.0,REG,00-0022921,00-0029638,00-0027691,00-0026221,NaN,Larry Fitzgerald,Michael Floyd,Andre Roberts,Early Doucet,NaN,5.421323,0.432432,3.35954,1.92334,NaN,9.3542,1.108108,6.049506,...,4.449163,NaN,58.675676,20.891892,49.810811,26.0,NaN,0.938108,0.333784,0.798378,0.416216,NaN,14.616446,8.0,12.080107,11.197194,NaN,8.471112,3.121951,6.708581,6.211461,NaN
4,ARI,2012,5.0,REG,00-0027691,00-0026221,00-0022921,00-0029638,NaN,Andre Roberts,Early Doucet,Larry Fitzgerald,Michael Floyd,NaN,4.079831,2.488967,6.123638,1.737143,NaN,6.854371,4.429398,10.891862,...,4.882286,NaN,56.462857,33.314286,63.548571,24.588571,NaN,0.850171,0.498057,0.960743,0.368971,NaN,15.123671,9.688256,12.262261,8.631579,NaN,9.00185,5.444024,6.894106,4.132283,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,WAS,2024,12.0,REG,00-0035659,00-0036626,00-0039355,00-0035208,00-0033591,Terry McLaurin,Dyami Brown,Luke McCaffrey,Olamide Zaccheaus,Noah Brown,3.658137,1.242214,0.664536,1.631636,2.609159,5.116495,1.892962,1.427217,...,3.488977,6.671457,50.121439,21.625846,24.945348,23.968566,46.947727,0.766987,0.328333,0.37946,0.363285,0.710012,17.206088,6.935652,12.915367,11.350867,12.890918,12.301826,4.551365,6.013613,6.922608,6.640311
7019,WAS,2024,13.0,REG,00-0035659,00-0035208,00-0039355,00-0033591,00-0036626,Terry McLaurin,Olamide Zaccheaus,Luke McCaffrey,Noah Brown,Dyami Brown,3.996936,1.472409,0.751146,3.507589,1.433543,5.339566,2.253026,1.316919,...,8.374751,3.193318,51.858168,23.976526,23.152211,48.028262,22.225285,0.777847,0.357388,0.348618,0.715342,0.331278,18.214877,10.777176,11.225894,12.411588,8.367331,13.634761,7.043155,6.403038,6.831436,6.247401
7020,WAS,2024,15.0,REG,00-0035659,00-0036626,00-0039355,00-0035208,NaN,Terry McLaurin,Dyami Brown,Luke McCaffrey,Olamide Zaccheaus,NaN,5.005143,2.079929,1.070957,1.856693,NaN,6.009619,2.695717,1.491845,...,3.399494,NaN,55.419912,27.457587,23.369316,25.244502,NaN,0.783427,0.37881,0.333607,0.355523,NaN,14.555657,8.5

In [37]:
team_qb_stats.to_csv('data/team_qb_stats.csv')
team_receiver_stats.to_csv('data/team_receiver_stats.csv')
team_rb_stats.to_csv('data/team_rb_stats.csv')

In [38]:
qb_rush_stats =stack_players(agg.loc[(agg.position.isin(['QB'])) & (~agg.week.isna())],2,  ['team_name','season','week', 'game_type'], rush_stat_cols)
qb_rush_stats = qb_rush_stats.fillna(0)
te_rush_stats =stack_players(agg.loc[(agg.position.isin(['TE'])) & (~agg.week.isna())],1,  ['team_name','season','week', 'game_type'], rush_stat_cols)
te_rush_stats = te_rush_stats.fillna(0)
wr_rush_stats =stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],1,  ['team_name','season','week', 'game_type'], rush_stat_cols)
wr_rush_stats = wr_rush_stats.fillna(0)

C:\Users\Sharedw\AppData\Local\Temp\ipykernel_17764\4220847580.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  qb_rush_stats = qb_rush_stats.fillna(0)
C:\Users\Sharedw\AppData\Local\Temp\ipykernel_17764\4220847580.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  te_rush_stats = te_rush_stats.fillna(0)
C:\Users\Sharedw\AppData\Local\Temp\ipykernel_17764\4220847580.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in

In [39]:
temp = team_rb_stats.merge( qb_rush_stats,  on=['team_name','season','week', 'game_type'])
temp = temp.merge( te_rush_stats,  on=['team_name','season','week', 'game_type'])
temp = temp.merge(wr_rush_stats,  on=['team_name','season','week', 'game_type'])
temp.to_csv('data/team_rushers.csv')

In [40]:
player_fantasy.loc[player_fantasy.full_name =='Will Shipley'].sort_values(by=['season','week'])[['season','week','offense_snaps', 'receptions', 'fantasy_points_ppr']]

,season,week,offense_snaps,receptions,fantasy_points_ppr
222834,2024,1.0,4.0,0.0,0.0
223650,2024,2.0,0.0,0.0,0.0
224463,2024,3.0,0.0,0.0,0.0
225271,2024,4.0,1.0,0.0,0.0
226889,2024,6.0,0.0,0.0,0.0
227691,2024,7.0,9.0,0.0,1.5
228493,2024,8.0,2.0,0.0,-0.3
229292,2024,9.0,1.0,0.0,0.0
230072,2024,10.0,11.0,0.0,3.4
230854,2024,11.0,0.0,0.0,0.0
